In [12]:
import pickle
import os

#### Make id2char, char2id

In [ ]:
# 데이터 경로 설정
scripts_path = "/graduate/graduate/Studio_project/10.한국어음성/scripts"
file_list = ["dev.trn", "eval_clean.trn", "eval_other.trn", "train.trn"]

# 모든 데이터를 담을 리스트
all_data = []

# 각 파일 읽기
for file_name in file_list:
    file_path = os.path.join(scripts_path, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        for line in lines:
            # "::" 구분자로 데이터 분리
            parts = line.strip().split("::")
            if len(parts) == 2:  # 두 부분으로 나뉜 경우만 처리
                audio_path = parts[0].strip()  # 음성 파일 경로
                transcript = parts[1].strip()  # 자막
                all_data.append((audio_path, transcript))

# 데이터 확인
print("총 데이터 수:", len(all_data))
print("예시 데이터:", all_data[5])


총 데이터 수: 628545
예시 데이터: ('KsponSpeech_05/KsponSpeech_0621/KsponSpeech_620006.pcm', '다 뿌신다고')


In [13]:
# 모든 자막 데이터를 합쳐서 문자 집합 생성
all_transcripts = [transcript for _, transcript in all_data]
all_text = "".join(all_transcripts)  # 모든 자막 문자열 합치기
char_set = sorted(set(all_text))    # 고유 문자 집합 생성 (정렬 포함)

# char2id와 id2char 생성
char2id = {char: idx for idx, char in enumerate(char_set)}
id2char = {idx: char for idx, char in enumerate(char_set)}

# 결과 확인
print("총 문자 수:", len(char_set))
print("char2id 예시:", list(char2id.items())[:10])
print("id2char 예시:", list(id2char.items())[:10])

# char2id와 id2char 저장 (Pickle 파일로 저장)
save_path = "/graduate/graduate/Studio_project/korean-speech-recognition-quartznet/datasets/meta/aihub"
os.makedirs(save_path,exist_ok=True)
with open(f"{save_path}/char2id.pkl", "wb") as f:
    pickle.dump(char2id, f)
with open(f"{save_path}/id2char.pkl", "wb") as f:
    pickle.dump(id2char, f)

print("char2id와 id2char 저장 완료")


총 문자 수: 2350
char2id 예시: [(' ', 0), ('!', 1), ('%', 2), ('&', 3), ('(', 4), (')', 5), ('*', 6), ('+', 7), (',', 8), ('-', 9)]
id2char 예시: [(0, ' '), (1, '!'), (2, '%'), (3, '&'), (4, '('), (5, ')'), (6, '*'), (7, '+'), (8, ','), (9, '-')]
char2id와 id2char 저장 완료


#### Make train data

In [17]:
base_path = "/graduate/graduate/Studio_project/10.한국어음성/KsponSpeech_01"

# train_data를 저장할 리스트
train_data = []

# KsponSpeech_01의 모든 하위 폴더 순회
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".txt"):  # 자막 파일만 처리
            txt_path = os.path.join(root, file)
            pcm_path = txt_path.replace(".txt", ".pcm")  # 동일 이름의 .pcm 파일 경로

            # 자막 읽기
            with open(txt_path, "r", encoding="cp949") as f:
                transcript = f.read().strip()  # 자막 파일에서 텍스트 읽기

            # train_data에 추가
            train_data.append((pcm_path, transcript))

# train_data를 pickle 파일로 저장
with open(f"{save_path}/train_data.pkl", "wb") as f:
    pickle.dump(train_data, f)


#### Make test data

In [20]:
test_pcm_path = "/graduate/graduate/Studio_project/10.한국어음성/eval/eval_clean"

# test_data를 저장할 리스트
test_data = []

# .pcm 파일만 추가
for root, dirs, files in os.walk(test_pcm_path):
    for file in files:
        if file.endswith(".pcm"):
            pcm_path = os.path.join(root, file)
            test_data.append((pcm_path, ""))  # 자막이 없으므로 빈 문자열로 대체

# test_data를 pickle 파일로 저장
with open(f"{save_path}/test_data.pkl", "wb") as f:
    pickle.dump(test_data, f)


#### Playground

In [2]:
path = '/graduate/graduate/Studio_project/10.한국어음성/scripts/train.trn'
# 파일 읽기
with open(path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 출력 확인
for line in lines[:4]:
    print(line.strip())  # 각 줄의 공백 제거 후 출력

KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000001.pcm :: 아/ 몬 소리야, 그건 또. b/
KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000002.pcm :: 나는 악습은 원래 없어진다+ 없어져야 된다고 생각하긴 했는데 근데 그/ 약간 필요악으로 하나 정도쯤은 있어야 되거든. 물 뜨러 가고.
KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000003.pcm :: b/ n/ 그래서 지호랑 계단 n/ 올라와서 b/ 막 위에 운동하는 기구 있대요. b/ 그서 그걸로 운동 할려구요. b/ n/
KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000004.pcm :: 뭐/ 정신과 병원도 그 약 타서 먹어보고, 그 한동안 연락이 안 된 적이 있었단 말이야. 그때가 언제였 언제였더라?


In [3]:
len(lines)

620000

In [5]:
sample_path = '/graduate/graduate/Studio_project/10.한국어음성/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000001.txt'
with open(sample_path, 'r', encoding='cp949') as file:
    lines = file.readlines()

# 내용 출력
for i, line in enumerate(lines):
    print(f"Line {i+1}: {line.strip()}")

Line 1: 아/ 몬 소리야, 그건 또. b/
